In [112]:
import numpy as np
import pickle
import pandas as pd
import os
from os.path import join
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
from scipy import stats
import xgboost as xgb
from sklearn.model_selection import KFold
from joblib import Parallel, delayed
from hyperopt import fmin, tpe, hp, Trials

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as mpl
plt.style.use('CCB_plot_style_0v4.mplstyle')
c_styles      = mpl.rcParams['axes.prop_cycle'].by_key()['color']   # fetch the defined color styles
high_contrast = ['#004488', '#DDAA33', '#BB5566', '#000000']


Bad key text.latex.preview in file CCB_plot_style_0v4.mplstyle, line 55 ('text.latex.preview  : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file CCB_plot_style_0v4.mplstyle, line 63 ('mathtext.fallback_to_cm : True ## When True, use symbols from the Computer Modern fonts')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution


## Loading training and test data:

In [113]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat.pkl"))


data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3391, 869)

In [114]:
train_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"), allow_pickle = True))

## 1. Training a model with only sequence information (ESM-1b):

#### (a) Creating input matrices:

In [15]:
train_ESM1b = np.array(list(data_train["ESM1b"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [9]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

100%|██████████| 200/200 [25:13<00:00,  7.57s/trial, best loss: 0.03743723040132387]


#### (c) Training and validating model:

In [16]:
param = {'learning_rate': 0.051447544749765035,
         'max_delta_step': 2.956459783615207,
         'max_depth': 5.034202474908222,
         'min_child_weight': 7.457989829577018,
         'num_rounds': 297.50601395689256,
         'reg_alpha': 1.0858835704466614, 
         'reg_lambda': 1.1385559144302175}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [17]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b.npy"), np.array(R2))

[-0.10233607311660751, -0.15314606513040566, -0.0455418650437268, -0.031018786560747122, -0.04521997323856698]
[1.8346361248595169, 1.6485991485910756, 1.6585845403859683, 1.6521416228683834, 1.5457048408922371]
[-0.14527321350278588, -0.28902782085496215, -0.178978678895795, -0.13075785627107317, -0.16639229744573591]


In [18]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b.npy"), test_Y)

-0.077 1.6 -0.171


In [19]:
y_test_pred_esm1b = y_test_pred

## 2. Training a model with only sequence information (ESM-1b_ts):

#### (a) Creating input matrices:

In [20]:
train_ESM1b = np.array(list(data_train["ESM1b_ts"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b_ts"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [11]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [21]:
param = {'learning_rate': 0.2831145406836757,
        'max_delta_step': 0.07686715986169101, 
        'max_depth': 4.96836783761305,
        'min_child_weight': 6.905400087083855,
        'num_rounds': 313.1498988074061,
        'reg_alpha': 1.717314107718892,
        'reg_lambda': 2.470354543039016}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [22]:
R2 = []
MSE = []
Pearson = []
y_valid_pred_esm1b_ts = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_esm1b_ts.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b_ts.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts.npy"), np.array(R2))

[0.02486240565545573, -0.1420797075589647, -0.10379250527068316, -0.10016291955822776, 0.020334890011688145]
[1.7813953024416926, 1.7386608227826128, 1.7782258113705949, 1.8222451562799455, 1.568066049170657]
[-0.11203758331226976, -0.35944639636185927, -0.26402379065961523, -0.24718002258067173, -0.18326611475387966]


In [23]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts.npy"), test_Y)

-0.1 1.689 -0.236


In [24]:
y_test_pred_esm1b_ts = y_test_pred

#### (d) Training model with test and train data for production mode:

In [25]:
train_ESM1b = np.array(list(data_train["ESM1b_ts"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b_ts"]))
test_Y = np.array(list(data_test["log10_kcat"]))

train_X = np.concatenate([train_ESM1b, test_ESM1b])
train_Y = np.concatenate([train_Y, test_Y])

In [26]:
param = {'learning_rate': 0.2831145406836757,
         'max_delta_step': 0.07686715986169101, 
         'max_depth': 4.96836783761305,
          'min_child_weight': 6.905400087083855,
           'num_rounds': 313.1498988074061,
            'reg_alpha': 1.717314107718892,
             'reg_lambda': 2.470354543039016}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(Pearson, MSE_dif_fp_test, R2_dif_fp_test)

pickle.dump(bst, open(join("..", "..", "data", "training_results", "saved_models",
                          "xgboost_sequence_only_train_and_test.pkl"), "wb"))

(0.9669336684582881, 0.0) 0.12759116635142767 0.906604299121613


## 3. Training a model with only reaction information (DRFP):

In [27]:
train_X = np.array(list(data_train["DRFP"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["DRFP"]))
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [28]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [29]:
param = {'learning_rate': 0.08987247189322463,
         'max_delta_step': 1.1939737318908727,
         'max_depth': 11.268531225242574,
         'min_child_weight': 2.8172720953826302,
         'num_rounds': 109.03643430746544,
         'reg_alpha': 1.9412226989868904,
         'reg_lambda': 4.950543905603358}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [30]:
R2 = []
MSE = []
Pearson = []
y_valid_pred_DRFP = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_DRFP.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_DRFP.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_DRFP.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_DRFP.npy"), np.array(R2))

[0.5881133955099471, 0.5462132054748297, 0.5819635238384331, 0.5726775753682218, 0.5511282628249307]
[1.049007141620355, 0.9050367902176091, 0.9322238709756645, 0.9829545115130323, 0.9227964694050949]
[0.3451563698153438, 0.29235824094365936, 0.3373433544833062, 0.32724681657693344, 0.30365573973170834]


In [31]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_DRFP.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_DRFP.npy"), test_Y)

0.559 0.945 0.308


In [32]:
y_test_pred_drfp = y_test_pred

#### (d) Training model with test and train data for production mode:

In [33]:
train_DRFP = np.array(list(data_train["DRFP"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_DRFP = np.array(list(data_test["DRFP"]))
test_Y = np.array(list(data_test["log10_kcat"]))

train_X = np.concatenate([train_DRFP, test_DRFP])
train_Y = np.concatenate([train_Y, test_Y])

In [34]:
param = {'learning_rate': 0.08987247189322463,
         'max_delta_step': 1.1939737318908727,
         'max_depth': 11.268531225242574,
         'min_child_weight': 2.8172720953826302,
         'num_rounds': 109.03643430746544,
         'reg_alpha': 1.9412226989868904,
         'reg_lambda': 4.950543905603358}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(Pearson, MSE_dif_fp_test, R2_dif_fp_test)

pickle.dump(bst, open(join("..", "..", "data", "training_results", "saved_models",
                          "xgboost_reaction_only_train_and_test.pkl"), "wb"))

(0.8334966832772743, 3.3104605748123116e-229) 0.4721235730299549 0.6544093665317637


## 4. Training a model with only reaction information (difference fingerprint):

#### (a) Creating input matrices:

In [35]:
train_X = np.array(list(data_train["difference_fp"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [27]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [36]:
param = {'learning_rate': 0.14154883958006167,
         'max_delta_step': 0.02234358170535966,
         'max_depth': 10.869653004093198,
         'min_child_weight': 1.7936882442746056,
         'num_rounds': 361.6168542774665,
         'reg_alpha': 4.825525325323308, 
         'reg_lambda': 2.74944090578774}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [37]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_diff_fp.npy"), np.array(R2))

[0.5782422525621318, 0.527702418863558, 0.6225643830666265, 0.5591353626322173, 0.5493683223243451]
[1.0982538548617198, 0.9290581544463321, 0.887808524875404, 1.0179675533983907, 0.9407644513087896]
[0.3144140657888673, 0.27357610896683715, 0.3689153032100193, 0.303283209803877, 0.29009706077912356]


In [38]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_diff_fp.npy"), test_Y)

0.519 1.001 0.267


In [39]:
y_test_pred_diff_fp = y_test_pred

## 5. Training a model with only reaction information (structural fingerprint):

#### (a) Creating input matrices:

In [41]:
train_X = ();
for ind in data_train.index:
    train_X = train_X + (np.array(list(data_train["structural_fp"][ind])).astype(int), )
train_X = np.array(train_X)
train_Y = np.array(list(data_train["log10_kcat"]))


test_X = ();
for ind in data_test.index:
    test_X = test_X + (np.array(list(data_test["structural_fp"][ind])).astype(int), )
test_X = np.array(test_X)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [33]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [42]:
param = {'learning_rate': 0.01126910440903659,
         'max_delta_step': 0.5777120839605732,
         'max_depth': 5.486901609313889,
         'min_child_weight': 6.14467742389769,
         'num_rounds': 488.943459090126,
         'reg_alpha': 4.629840853377147,
         'reg_lambda': 2.1047561335691745}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [43]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_str_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_str_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_str_fp.npy"), np.array(R2))

[0.5461680032633288, 0.5239731776116427, 0.5969058521176137, 0.5601464413573072, 0.5274004514654467]
[1.1329108964865013, 0.9326878465396904, 0.9177482860266996, 1.0106627733905318, 0.9644893012025042]
[0.292779377092681, 0.27073807881652767, 0.3476330958885909, 0.30828274329900063, 0.2721942364869312]


In [44]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_str_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_str_fp.npy"), test_Y)

0.524 0.992 0.274


## 6. Training a model with enzyme and reaction information (ESM1b_ts/DRFP):

In [45]:
train_X = np.array(list(data_train["DRFP"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["DRFP"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [38]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 6,14),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [46]:
param = {'learning_rate': 0.05221672412884108,
         'max_delta_step': 1.0767235463496743,
         'max_depth': 11.329014411591299,
         'min_child_weight': 14.724796449973605,
         'num_rounds': 298.9598325756988,
         'reg_alpha': 2.8295816318634452,
         'reg_lambda': 0.6528469146574993}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [47]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(R2))

[0.431459139611596, 0.3200005154441318, 0.408405900279767, 0.34895902652111643, 0.3510372955800634]
[1.3037907166297578, 1.1896381263532094, 1.1765466577947605, 1.2925965723906208, 1.1726797961960855]
[0.18610749917289393, 0.06983050250293021, 0.16367035234554095, 0.11532176843159525, 0.11509322771881669]


In [48]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_DRFP.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_DRFP.npy"), test_Y)

0.33 1.237 0.095


In [49]:
y_test_pred_esm1b_ts_drfp = y_test_pred

## 7. Training a model with enzyme and reaction information (ESM1b_ts/diff_fp):

#### (a) Creating input matrices:

In [63]:
train_X = np.array(list(data_train["difference_fp"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [59]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    R2 = []
    skf = KFold(n_splits=5)

    def train_and_evaluate(train_index, test_index):
        dtrain = xgb.DMatrix(train_X[train_index], train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])

        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        mse = np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2)
        r2 = r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred)
        return mse, r2
    
    results = []
    for train_index, test_index in skf.split(train_X, train_Y):
        result = train_and_evaluate(train_index, test_index)
        results.append(result)

    MSE, R2 = zip(*results)
    return -np.mean(R2)

space = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    "gamma": hp.uniform("gamma", 0, 0.5),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space=space, algo=tpe.suggest, max_evals = 200, trials=trials)

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 200/200 [1:14:30<00:00, 22.35s/trial, best loss: -0.4463204621674697]


#### (c) Training and validating model:

In [60]:
best

{'colsample_bytree': 0.7706304470135901,
 'gamma': 0.08473280513934948,
 'learning_rate': 0.08292031202222186,
 'max_delta_step': 2.0905240205907973,
 'max_depth': 8.018890262009021,
 'min_child_weight': 1.152288086135032,
 'num_rounds': 188.30718240661182,
 'reg_alpha': 2.097680263149934,
 'reg_lambda': 0.09212019804805793}

In [62]:
'''param = {'learning_rate': 0.5727401435817077, 
         'max_delta_step': 0.022572978136953803,
         'max_depth': 9.734956573895278,
         'min_child_weight': 2.026404280518698,
         'num_rounds': 259.69265795096726,
         'reg_alpha': 7.333074414515098,
         'reg_lambda': 0.8545111451043885}''';

'''param = {'learning_rate': 0.029815704092754386,
        'max_delta_step': 1.506753151302061,
        'max_depth': 7.916290092522673,
        'min_child_weight': 9.004351370580997,
        'num_rounds': 162.51704178700743,
        'reg_alpha': 2.415815193301269,
        'reg_lambda': 4.548495397904814}''';

param = {'colsample_bytree': 0.7706304470135901,
        'gamma': 0.08473280513934948,
        'learning_rate': 0.08292031202222186,
        'max_delta_step': 2.0905240205907973,
        'max_depth': 8.018890262009021,
        'min_child_weight': 1.152288086135032,
        'num_rounds': 188.30718240661182,
        'reg_alpha': 2.097680263149934,
        'reg_lambda': 0.09212019804805793}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))
param["tree_method"] = "gpu_hist"

del param["num_rounds"]

In [65]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))
# 0.395 1.16 0.151 old old params
# 0.368 1.098 0.119 old params
# 0.424 1.024 0.179 new params
# 0.425 1.028 0.175 new new params

#np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_diff_fp.npy"), bst.predict(dtest))
#np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_diff_fp.npy"), test_Y)

0.425 1.028 0.175


In [40]:
y_test_pred_esm1b_ts_drfp = y_test_pred

#### (d) Training model with test and train data for production mode:

In [66]:
# train on all the data
train_X = np.concatenate([train_X, test_X])
train_Y = np.concatenate([train_Y, test_Y])

In [67]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0], 3), np.round(MSE_dif_fp_test, 3), np.round(R2_dif_fp_test, 3))
# 0.989 0.033 0.973 old old params
# 0.982 0.053 0.957 old params
# 0.904 0.348 0.721 new params
# 0.98 0.071 0.943 new new params

#pickle.dump(bst, open(join("..", "..", "data", "training_results", "saved_models", "xgboost_train_and_test.pkl"), "wb"))

0.98 0.071 0.943


## 8. Model with enzyme and reaction information (ESM1b_ts/DRFP [mean]):

#### Cross-Validation

In [57]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    y_valid_pred = np.mean([y_valid_pred_DRFP[i], y_valid_pred_esm1b_ts[i]], axis =0)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_DRFP_mean.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_DRFP_mean.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_DRFP_mean.npy"), np.array(R2))

[0.48885823200781836, 0.354974544097536, 0.455574304946437, 0.4236548592696329, 0.43460493901700165]
[1.2485969063138214, 1.1331299043335048, 1.1332844703900258, 1.2103006866579678, 1.0815445450679488]
[0.22056228377539155, 0.11401387500600313, 0.19442259638895099, 0.17164667305413162, 0.18386409013022098]


#### Validation on test set:

In [58]:
y_test_pred = np.mean([y_test_pred_drfp, y_test_pred_esm1b_ts], axis =0)

MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)
print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

0.424 1.127 0.175


In [59]:
np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_DRFP_mean.npy"), y_test_pred)
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_DRFP_mean.npy"), test_Y)

## 9. Training a model with enzyme and reaction information (ESM2_3B/diff_fp):

#### (a) Creating input matrices:

In [119]:
train_X = np.array(list(data_train["difference_fp"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM2_3B"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM2_3B"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [116]:
def train_and_evaluate(train_index, test_index):
    dtrain = xgb.DMatrix(train_X[train_index], train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    y_valid_pred = bst.predict(dvalid)
    r2 = r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred)
    return r2

def cross_validation_mse_gradient_boosting(param):
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    R2 = []
    kf = KFold(n_splits=5)
    
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        result = train_and_evaluate(train_index, test_index)
        R2.append(result)

    return -np.mean(R2)

space = {
    "learning_rate": hp.uniform("learning_rate", 0.001, 1),
    "max_depth": hp.quniform("max_depth", 1, 20, 1),
    "gamma": hp.uniform("gamma", 0, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.1, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 10),
    "reg_alpha": hp.uniform("reg_alpha", 0, 10),
    "max_delta_step": hp.uniform("max_delta_step", 0, 10),
    "min_child_weight": hp.uniform("min_child_weight", 0.01, 20),
    "num_rounds":  hp.quniform("num_rounds", 10, 300, 10)
}

trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space=space, algo=tpe.suggest, max_evals = 200, trials=trials)

100%|██████████| 200/200 [3:36:58<00:00, 65.09s/trial, best loss: -0.3579976419781222]  


#### (c) Training and validating model:

In [117]:
best

{'colsample_bytree': 0.9965828569106562,
 'gamma': 0.9950146409429289,
 'learning_rate': 0.7886969584173175,
 'max_delta_step': 8.387703310583698,
 'max_depth': 3.0,
 'min_child_weight': 18.34298401258303,
 'num_rounds': 290.0,
 'reg_alpha': 7.564932934795392,
 'reg_lambda': 8.456788423168604}

In [120]:
'''param = {'learning_rate': 0.5727401435817077, 
        'max_delta_step': 0.022572978136953803,
        'max_depth': 9.734956573895278,
        'min_child_weight': 2.026404280518698,
        'num_rounds': 259.69265795096726,
        'reg_alpha': 7.333074414515098,
        'reg_lambda': 0.8545111451043885}''';

'''param = {'colsample_bytree': 0.5033789302958038,
        'gamma': 0.0037752141604097122,
        'learning_rate': 0.061921864701214496,
        'max_delta_step': 0.9351742448381885,
        'max_depth': 10.7625991614889,
        'min_child_weight': 13.939420814219764,
        'num_rounds': 144.57893540285642,
        'reg_alpha': 2.5404285747350235,
        'reg_lambda': 2.0705777927656057}''';

'''param = {'colsample_bytree': 0.3404528115480022,
        'gamma': 0.6219875598661728,
        'learning_rate': 0.984051033355358,
        'max_delta_step': 6.438638552418567,
        'max_depth': 10.0,
        'min_child_weight': 3.6623047466798453,
        'num_rounds': 160.0,
        'reg_alpha': 6.137824021103583,
        'reg_lambda': 4.284160185624403}''';

param = {'colsample_bytree': 0.9965828569106562,
        'gamma': 0.9950146409429289,
        'learning_rate': 0.7886969584173175,
        'max_delta_step': 8.387703310583698,
        'max_depth': 3.0,
        'min_child_weight': 18.34298401258303,
        'num_rounds': 290.0,
        'reg_alpha': 7.564932934795392,
        'reg_lambda': 8.456788423168604}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))
param["tree_method"] = "gpu_hist"

del param["num_rounds"]

In [121]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    y_valid_pred = bst.predict(dvalid)

    MSE.append(np.round(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2),3))
    R2.append(np.round(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred),3))
    Pearson.append(np.round(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0],3))

for i in range(5):
    print(f"({Pearson[i]}, {MSE[i]}, {R2[i]})")

(0.41, 1.346, 0.046)
(0.515, 1.16, 0.154)
(0.476, 1.302, 0.157)
(0.444, 1.286, 0.024)
(0.485, 1.359, 0.136)


In [122]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
y_test_pred = bst.predict(dtest)

MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))
# 0.607 0.789 0.367 old params
# 0.586 0.821 0.342
# 0.966 0.086 0.931
# 0.483 1.079 0.135

#np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM2_3B_diff_fp.npy"), bst.predict(dtest))
#np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM2_3B_diff_fp.npy"), test_Y)

0.483 1.079 0.135


In [67]:
y_test_pred_ESM2_3B_drfp = y_test_pred

#### (d) Training model with test and train data for production mode:

In [123]:
train_X = np.concatenate([train_X, test_X])
train_Y = np.concatenate([train_Y, test_Y])

In [124]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))
# 0.986 0.037 0.971 old params
# 0.987 0.036 0.971
# 0.983 0.041 0.967
# 0.948 0.131 0.895

#pickle.dump(bst, open(join("..", "..", "data", "training_results", "saved_models", "xgboost_train_and_test_ESM2_3B.pkl"), "wb"))

0.948 0.131 0.895


## 9. Training a model with enzyme rep., reaction information, and additional features (ESM1b_ts/diff_fp/flux/KM):

Loading dataset with additonal features:

In [54]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_with_KM_and_flux.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_with_KM_and_flux.pkl"))

data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3421, 850)

In [ ]:
train_X = ();
train_X = np.array(list(data_train["DRFP"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"])),
                          np.reshape(np.array(list(data_train["KM"])), (-1,1)),
                          np.reshape(np.array(list(data_train["flux"])), (-1,1))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = ();
test_X = np.array(list(data_test["DRFP"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"])),
                          np.reshape(np.array(list(data_test["KM"])), (-1,1)),
                          np.reshape(np.array(list(data_test["flux"])), (-1,1))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

In [ ]:
param = {'learning_rate': 0.15055870206296312,
         'max_delta_step': 0.13821534396910307,
         'max_depth': 5.338955142881738,
         'min_child_weight': 14.84613730467497,
         'num_rounds': 294.13028718637383,
         'reg_alpha': 2.6752278199969153,
         'reg_lambda': 0.6063171152564584}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_diff_fp_flux_KM.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_diff_fp_flux_KM.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_diff_fp_flux_KM.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(Pearson[0], MSE_dif_fp_test, R2_dif_fp_test)


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_diff_fp_flux_KM.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_diff_fp_flux_KM.npy"), test_Y)